In [216]:
# R_challenge
# merge all the data into one data table
import os
import csv
import numpy as np
import pandas as pd
# import county list
directory = "./all_counties"
f1 = open(directory + "/county_name.txt","r")
rawdata = f1.readlines()
f1.close()

county_str = map(lambda x: x.splitlines(), rawdata)
county_str = [county_str[i][0] for i in range(len(county_str))]

In [217]:
# define a standard time zone:
f = open(directory+"/anderson.csv","r")
ff = list(csv.reader(f))
labels=['Date','owop','owgp','gwgp','gwop']
SD = pd.DataFrame.from_records(ff[8:len(ff)-1],columns=labels)
SD.Date=pd.to_datetime(SD.Date)
SD = pd.DataFrame(SD.Date)
SD.columns=['SD'] # SD - standard date format
SD.set_index(SD.SD, inplace=True)

In [218]:
directory1 = "./pd_merged"
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        county_name = str(filename.split(".")[0])
        if county_name in county_str:
            county_str = filter(lambda x: x != county_name, county_str) # keep track of the remaining counties
            with open(directory + "/" + filename) as ff:
                ffreader = list(csv.reader(ff))
            county_name_1 = str(ffreader[4][0].split(": ")[1])
            if county_name.upper() != county_name_1:
                notc += 1
                print "inconsistent county name between filename and content!"
            else:
                df = pd.DataFrame.from_records(ffreader[8:len(ffreader)-1],columns=labels)
                df.Date = pd.to_datetime(df.Date)
                df.owgp = pd.to_numeric(df.owgp)
                df.owop = pd.to_numeric(df.owop)
                df.gwgp = pd.to_numeric(df.gwgp)
                df.gwop = pd.to_numeric(df.gwop)
                df.set_index(df.Date, inplace=True)
                pd_df = pd.concat([SD,df],axis=1,join='outer')
                pd_df.drop('Date',axis=1,inplace=True)
                pd_df.fillna(1.0,inplace=True)
                pd_df.replace(0.0,1.0,inplace=True)
                pd_df['county']=county_name.upper()
                pd_df.to_csv(directory1+"/pd_"+county_name+".csv",index=False)

In [219]:
# merged data together
df1 = pd.read_csv(directory1+'/pd_andrews.csv')
for filename in os.listdir(directory1):
    if filename != 'pd_andrews.csv':
        df2 = pd.read_csv(directory1+'/'+filename)
        df1 = pd.concat([df1,df2],axis=0)

df1.to_csv("merged.csv",index=False)